In [73]:
import pandas as pd
import numpy as np
import requests
import json
import os
from dotenv import load_dotenv
from tqdm.notebook import tqdm

In [74]:
#config the environment variables
load_dotenv()
loc_url=os.getenv("ONEMAP_API_URL")
loc_key=os.getenv("ONEMAP_API_ACCESS_TOKEN")

#define the function to get the longtitude and latitude of the hdb with the address
def get_pos_hdb(address:str)->tuple[str,str]:

    params={
        "searchVal":address,
        "returnGeom":"Y",
        "getAddrDetails":"Y",
        "pageNum":1
    }

    headers={
        "Authorization":f"{loc_key}"
    }

    response=requests.get(loc_url,params=params,headers=headers)

    if response.status_code==200:
        data=response.json()
        lon=float(data["results"][0]["LONGITUDE"])
        lat=float(data["results"][0]["LATITUDE"])
        postal_code = data["results"][0]["POSTAL"]
        print(data)
        return lon,lat,postal_code
    else:
        print(f"Error: {response.status_code,}")
        return None,None
    
#To test if the function is working
# test_address="woodlands drive 62 681B"
# get_pos_hdb(test_address)


In [ ]:
import time
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor, as_completed

SESSION = requests.Session()

@lru_cache(maxsize=None)
def _get_pos_cached(address: str):
    params = {"searchVal": address, "returnGeom":"Y", "getAddrDetails":"Y", "pageNum":1}
    headers = {"Authorization": f"{os.getenv('ONEMAP_API_ACCESS_TOKEN')}"}
    backoff = 0.5
    for _ in range(6):
        resp = SESSION.get(os.getenv('ONEMAP_API_URL'), params=params, headers=headers, timeout=10)
        if resp.status_code == 200:
            data = resp.json()
            lon = float(data["results"][0]["LONGITUDE"])
            lat = float(data["results"][0]["LATITUDE"])
            postal_code = data["results"][0]["POSTAL"]
            return lon, lat, postal_code
        if resp.status_code in (429, 503):
            time.sleep(backoff)
            backoff = min(backoff * 2, 8.0)
            continue
        break
    return None, None

def get_hdb_coordinates_and_postal_code_df_concurrent(df: pd.DataFrame, max_workers: int = 12, qps: float = 8.0):
    addresses = (df['STREET'].astype(str) + ' ' + df['BLOCK'].astype(str))
    uniq = addresses.drop_duplicates().tolist()
    #the combination of Street and block still has duplications so when using api query still need to remove the duplicates within the method
    
    min_interval = 1.0 / max(qps, 1e-6)
    last = [0.0]
    lock = __import__('threading').Lock()

    def worker(addr):
        with lock:
            now = time.time()
            wait = min_interval - (now - last[0])
            if wait > 0: time.sleep(wait)
            last[0] = time.time()
        return addr, _get_pos_cached(addr)

    results = {}
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(worker, a) for a in uniq]
        for fut in tqdm(as_completed(futures),total=len(uniq), desc="Getting HDB coordinates"):
            addr, (lon, lat, postal_code) = fut.result()
            results[addr] = (lon, lat, postal_code)

    out = addresses.map(results.get).apply(pd.Series)
    out.columns = ['LONGITUDE', 'LATITUDE', 'POSTAL']
    return out

In [76]:

def haversine_matrix(hdb_lat:np.ndarray,hdb_lon:np.ndarray,mall_lat:np.ndarray,mall_lon:np.ndarray):
    R=6371
    
    dlat=hdb_lat[:,None]-mall_lat[None,:] #为了便于计算，将hdb_lat和hdb_lon扩展为矩阵
    dlon=hdb_lon[:,None]-mall_lon[None,:]

    a=np.sin(dlat/2)**2+np.cos(hdb_lat)[:, None]*np.cos(mall_lat)[None, :]*np.sin(dlon/2)**2
    c=2*np.arctan2(np.sqrt(a),np.sqrt(1-a))
    return R*c


def create_auxiliary_location_features(hdb_coords:pd.DataFrame,
                        auxilliary_df:pd.DataFrame,
                        radii=(1.0,2.0,5.0),
                        batch_size:int |None=None)->pd.DataFrame: #hdb_df is the dataframe got from get_hdb_coordinates_df
    mlat=np.radians(auxilliary_df['LATITUDE'].to_numpy())
    mlon=np.radians(auxilliary_df['LONGITUDE'].to_numpy())
    hlat=np.radians(hdb_coords['LATITUDE'].to_numpy())
    hlon=np.radians(hdb_coords['LONGITUDE'].to_numpy())

    #create a empty container to store the mall features
    n=len(hlat)
    nearest=np.full((n,),np.inf,dtype=np.float32)
    counts=[np.zeros(n,dtype=np.int32) for _ in radii]

    if batch_size is None:
        dist=haversine_matrix(hlat,hlon,mlat,mlon)
        nearest=dist.min(axis=1).astype(np.float32)
        for i,r in enumerate(radii):
            counts[i]=np.array((dist<r).sum(axis=1),dtype=np.int32)
    else:
       for i in tqdm(range(0, n, batch_size),total=(n+batch_size-1)//batch_size, desc="Creating mall features"):
            j = min(i+batch_size, n)
            dist = haversine_matrix(hlat[i:j], hlon[i:j], mlat, mlon)
            nearest[i:j] = dist.min(axis=1).astype(np.float32)
            for k, r in enumerate(radii):
                counts[k][i:j] = (dist <= r).sum(axis=1).astype(np.int32)
    output=pd.DataFrame({
        'NEAREST_MALL_KM': nearest,
        **{f'MALL_COUNT_{int(r)}KM': c for r, c in zip(radii, counts)}
    }, index=hdb_coords.index)
    
    return output
            


In [77]:
#load data
data_path="../Dataset/"
df_train=pd.read_csv(data_path+"train.csv")
df_shopping_malls=pd.read_csv(data_path+"/auxiliary-data/sg-shopping-malls.csv")


In [78]:
df_secondary_schools=pd.read_csv(data_path+"/auxiliary-data/sg-secondary-schools.csv")
df_primary_schools=pd.read_csv(data_path+"/auxiliary-data/sg-primary-schools.csv")
df_hawkers=pd.read_csv(data_path+"/auxiliary-data/sg-gov-hawkers.csv")
df_mrt_stations=pd.read_csv(data_path+"/auxiliary-data/sg-mrt-stations.csv")


In [79]:
df_train.shape

(162691, 11)

In [80]:
df_train.head()

,MONTH,TOWN,FLAT_TYPE,BLOCK,STREET,FLOOR_RANGE,FLOOR_AREA_SQM,FLAT_MODEL,ECO_CATEGORY,LEASE_COMMENCE_DATA,RESALE_PRICE
0,2020-10,woodlands,4 room,681B,woodlands drive 62,07 to 09,102.0,premium apartment,uncategorized,2000,420000.0
1,2021-07,bishan,4 room,264,bishan street 24,07 to 09,104.0,model a,uncategorized,1992,585000.0
2,2021-05,bukit panjang,4 room,520,jelapang road,19 to 21,102.0,model a,uncategorized,1998,450000.0
3,2021-08,punggol,4 room,121B,edgedale plains,16 to 18,93.0,model a,uncategorized,2017,465000.0
4,2023-05,hougang,5 room,997B,Buangkok Crescent,10 to 12,113.0,improved,uncategorized,2018,710000.0


In [81]:
df_train.duplicated().sum()

np.int64(121)

In [82]:
df_train[df_train.duplicated()]

,MONTH,TOWN,FLAT_TYPE,BLOCK,STREET,FLOOR_RANGE,FLOOR_AREA_SQM,FLAT_MODEL,ECO_CATEGORY,LEASE_COMMENCE_DATA,RESALE_PRICE
16061,2017-09,pasir ris,5-room,192,pasir ris street 12,10 to 12,128.0,improved,uncategorized,1993,468000.0
20599,2023-06,bedok,3 room,92,Bedok North Avenue 4,01 to 03,67.0,new generation,uncategorized,1979,350000.0
23225,2021-07,choa chu kang,4 room,815C,choa chu kang avenue 7,04 to 06,93.0,model a,uncategorized,2017,460000.0
32776,2019-12,punggol,4-room,312C,sumang link,10 to 12,92.0,model a,uncategorized,2015,470000.0
35606,2022-07,punggol,5 room,669A,edgefield plains,07 to 09,112.0,premium apartment,uncategorized,2016,738000.0
...,...,...,...,...,...,...,...,...,...,...,...
159008,2017-02,tampines,4-room,842H,tampines street 82,07 to 09,106.0,model a,uncategorized,1995,470000.0
160717,2019-02,yishun,4-room,348A,yishun avenue 11,13 to 15,92.0,dbss,uncategorized,2013,620000.0
160898,2017-09,marine parade,4-room,45,marine crescent,01 to 03,88.0,improved,uncategorized,1975,470000.0
161424,2017-11,pasir ris,5-room,140,pasir ris street 11,01 to 03,136.0,model a,uncategorized,1994,468000.0


In [85]:
print(f"secondary_schools dataset has {df_secondary_schools.duplicated().sum()} duplicated rows")
print(f"primary_schools dataset has {df_primary_schools.duplicated().sum()} duplicated rows")
print(f"hawkers dataset has {df_hawkers.duplicated().sum()} duplicated rows")
print(f"mrt_stations dataset has {df_mrt_stations.duplicated().sum()} duplicated rows")
print(f"shopping_malls dataset has {df_shopping_malls.duplicated().sum()} duplicated rows")

secondary_schools dataset has 0 duplicated rows
primary_schools dataset has 0 duplicated rows
hawkers dataset has 0 duplicated rows
mrt_stations dataset has 0 duplicated rows
shopping_malls dataset has 0 duplicated rows


In [86]:
columns_to_check=df_shopping_malls.select_dtypes(include=['object']).columns.tolist()
df_list=[df_shopping_malls,df_secondary_schools,df_primary_schools,df_hawkers,df_mrt_stations]
for i, df in enumerate(df_list):
    print(f"Checking {i+1} dataset")
    for col in columns_to_check:
        print(f"{col}: {df_shopping_malls[col].unique()}")
    

Checking 1 dataset
NAME: ['Bugis Junction' 'Bugis+' 'Capitol Piazza' 'Cathay Cineleisure Orchard'
 'The Centrepoint' 'City Square Mall' 'CityLink Mall' 'Duo'
 'Far East Plaza' 'Funan' 'Great World City' 'HDB Hub'
 'Holland Road Shopping Centre' 'ION Orchard' 'Junction 8' 'Liat Towers'
 'Lucky Plaza' 'Marina Bay Sands' 'Marina Bay Link Mall' 'Marina Square'
 'Millenia Walk' 'Mustafa Centre' 'Ngee Ann City' 'Orchard Central'
 'Orchard Gateway' 'Palais Renaissance' 'The Paragon'
 "People's Park Centre" "People's Park Complex" 'Plaza Singapura'
 'Raffles City' 'Shaw House and Centre' 'Sim Lim Square' 'The South Beach'
 'Square 2' 'Suntec City' 'Tanjong Pagar Centre' 'Tekka Centre'
 'Tiong Bahru Plaza' 'Thomson Plaza' 'Velocity @ Novena Square'
 'Wheelock Place' 'Wisma Atria' 'Bedok Mall' 'Century Square'
 'Changi City Point' 'Downtown East' 'Eastpoint Mall'
 'Jewel Changi Airport' 'Katong Shopping Centre' 'Kallang Wave Mall'
 'Leisure Park Kallang' 'i12 Katong' 'Our Tampines Hub' 'Parkway 

In [87]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162691 entries, 0 to 162690
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   MONTH                162691 non-null  object 
 1   TOWN                 162691 non-null  object 
 2   FLAT_TYPE            162691 non-null  object 
 3   BLOCK                162691 non-null  object 
 4   STREET               162691 non-null  object 
 5   FLOOR_RANGE          162691 non-null  object 
 6   FLOOR_AREA_SQM       162691 non-null  float64
 7   FLAT_MODEL           162691 non-null  object 
 8   ECO_CATEGORY         162691 non-null  object 
 9   LEASE_COMMENCE_DATA  162691 non-null  int64  
 10  RESALE_PRICE         162691 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 13.7+ MB


In [88]:
print(f"Check categorical features: {df_train.select_dtypes(include=['object']).columns}")

categorical_features=df_train.select_dtypes(include=['object']).columns.tolist()
for col in categorical_features:
    print(f"{col}: {df_train[col].unique()}")



Check categorical features: Index(['MONTH', 'TOWN', 'FLAT_TYPE', 'BLOCK', 'STREET', 'FLOOR_RANGE',
       'FLAT_MODEL', 'ECO_CATEGORY'],
      dtype='object')
MONTH: ['2020-10' '2021-07' '2021-05' '2021-08' '2023-05' '2024-03' '2020-07'
 '2025-03' '2025-07' '2022-07' '2019-08' '2024-10' '2019-11' '2018-01'
 '2020-11' '2022-11' '2017-10' '2022-12' '2021-11' '2018-02' '2018-05'
 '2022-05' '2017-05' '2025-06' '2024-06' '2017-09' '2023-08' '2024-07'
 '2024-11' '2019-04' '2018-11' '2023-09' '2019-05' '2023-04' '2017-04'
 '2021-09' '2022-09' '2023-11' '2021-06' '2018-07' '2020-06' '2024-02'
 '2023-03' '2022-04' '2024-08' '2022-02' '2017-02' '2025-02' '2021-01'
 '2018-03' '2018-09' '2019-10' '2025-04' '2019-06' '2019-12' '2021-10'
 '2024-04' '2022-03' '2021-03' '2020-03' '2024-01' '2019-02' '2019-01'
 '2021-04' '2020-08' '2017-06' '2020-01' '2019-09' '2022-06' '2021-12'
 '2020-09' '2017-07' '2018-10' '2021-02' '2024-12' '2017-08' '2025-01'
 '2019-03' '2017-11' '2023-12' '2024-05' '2023-02' '2

In [90]:
print(f"Before processing, the unique values of 'FLAT_TYPE' is: {df_train['FLAT_TYPE'].value_counts()}")
print(f"Before processing, the unique values of 'STREET' is: {df_train['STREET'].value_counts()}")


Before processing, the unique values of 'FLAT_TYPE' is: FLAT_TYPE
4 room              48459
5 room              27916
3 room              26702
4-room              20414
5-room              12167
3-room              11904
executive           11801
2 room               2503
2-room                700
multi generation       68
1 room                 37
1-room                 20
Name: count, dtype: int64
Before processing, the unique values of 'STREET' is: STREET
yishun ring road         1398
bedok reservoir road     1032
punggol drive            1020
punggol field             947
Yishun Ring Road          940
                         ... 
jurong west street 51       3
margaret drive              2
geylang east avenue 2       2
Margaret Drive              1
Seng Poh Road               1
Name: count, Length: 1145, dtype: int64


In [91]:
df_train['FLAT_TYPE']=(df_train['FLAT_TYPE']
                        .str.strip()
                        .str.replace(r'\s+', ' ', regex=True)
                        .str.replace(' room','-room',regex=False)
                        )
df_train['STREET']=df_train['STREET'].str.lower()

In [92]:
print(f"After processing, the shape of 'FLAT_TYPE' is: {df_train['FLAT_TYPE'].value_counts()}")
print(f"After processing, the shape of 'STREET' is: {df_train['STREET'].value_counts()}")

After processing, the shape of 'FLAT_TYPE' is: FLAT_TYPE
4-room              68873
5-room              40083
3-room              38606
executive           11801
2-room               3203
multi generation       68
1-room                 57
Name: count, dtype: int64
After processing, the shape of 'STREET' is: STREET
yishun ring road         2338
punggol drive            1713
bedok reservoir road     1691
punggol field            1559
ang mo kio avenue 10     1545
                         ... 
kreta ayer road             6
seng poh road               4
sembawang way               4
margaret drive              3
geylang east avenue 2       2
Name: count, Length: 574, dtype: int64


In [97]:
df_hdb_dedup=df_train.drop_duplicates(keep='first') #Remove the completely identical rows


In [98]:
df_hdb_coords=get_hdb_coordinates_and_postal_code_df_concurrent(df_hdb_dedup)


Getting HDB coordinates:   0%|          | 0/9631 [00:00<?, ?it/s]

In [99]:

df_hdb_coords.head(10)
outpath="../Dataset/EDA/"
df_hdb_coords.to_feather(outpath+"hdb_coords.feather")



In [100]:
mall_features=create_auxiliary_location_features(df_hdb_coords,df_shopping_malls,radii=(1.0,2.0,5.0),batch_size=50000)

Creating mall features:   0%|          | 0/4 [00:00<?, ?it/s]

In [101]:
mall_features.head(30)

,NEAREST_MALL_KM,MALL_COUNT_1KM,MALL_COUNT_2KM,MALL_COUNT_5KM
0,1.994705,0,1,5
1,1.176150,0,3,8
2,0.980479,1,1,3
3,0.463176,1,4,7
4,0.851707,1,2,9
5,0.493906,1,2,9
6,1.346054,0,1,3
7,1.589531,0,2,9
8,0.138148,2,3,6
9,1.162108,0,1,5


In [ ]:
df_new_train = (
    df_hdb_dedup
      .join(df_hdb_coords, how='left')        # Add coordinates and postal code
      .join(mall_features, how='left')        # Add shopping mall related features
)
df_new_train.shape

(162464, 18)

In [103]:
outpath="../Dataset/"
df_new_train.to_csv(outpath+"train_with_mall_features_and_hdb_coordinates_and_postal_code.csv",index=False)